# 調和励振系の数値シミュレーション

吉田勝俊（宇都宮大学）

## 参考情報

- [Pythonで運動方程式を解く(odeint) - Qiita](https://qiita.com/binaryneutronstar/items/ad5efa27fd626826846f)
- [[Python] Numpyの参照、抽出、結合 - Qiita](https://qiita.com/supersaiakujin/items/d63c73bb7b5aac43898a)
- [[Python/matplotlib] FuncAnimationを理解して使う - Qiita](https://qiita.com/osanshouo/items/3c66781f41884694838b)
- [2つの信号間の遅延を推定する - Qiita](https://qiita.com/inoory/items/3ea2d447f6f1e8c40ffa)

In [ ]:
import numpy as np                              #数値計算ライブラリ
from math import sin, cos                       #低機能だが高速な数学関数・定数
from numpy.linalg import norm                   #ベクトルのノルム
from scipy.integrate import odeint              #常微分方程式ライブラリ
import matplotlib.pyplot as plt                 #描画ライブラリ
from matplotlib import rc                       #グラフ調整ライブラリ
from matplotlib.animation import FuncAnimation  #アニメーションライブラリ
#Colab用の設定（グラフィックのインライン表示）
%matplotlib inline

## 調和励振系の運動方程式

In [ ]:
def org_funcs():
    '''
    調和励振系（原形 original）の外力と運動方程式
    '''
    def force(t, param):
        '''
        外力 f(t)
        '''
        m, c, k, P, Om = param
        ft = P*sin(Om*t)
        
        return ft 

    def eom(x, t, param):
        '''
        運動方程式
        '''
        m, c, k, P, Om = param
        return np.array([
            x[1],
            -(c/m)*x[1] - (k/m)*x[0] + force(t,param)
        ])

    return (force, eom)

In [ ]:
def Solve(funcs, param, x0, time, new_om=None):
    '''
    調和励振系の数値解を求める関数
    '''
    force, eom = funcs()     #外力と運動方程式の取得
    new_param = param.copy() #元のparamを書き換えぬよう複製
    
    if new_om is not None:
        new_param[-1] = new_om   #最終成分＝外力角振動数
        
    ###運動方程式を数値的に解く
    motion = odeint(
        eom,                #運動方程式を表すユーザ関数
        x0,                 #初期条件
        time,               #時間軸を表す数列
        args=(new_param,)   #運動方程式の係数
    )
    
    ###外力の数列を作る
    forces = np.array([
        force(t, new_param) for t in time
    ])

    return (forces, motion)

## 演習 5.2 ( 調和励振系の振動波形 )

In [ ]:
def draw_waves(axs, ts, forces, xs, 
               inlab=r'$f(t)$', outlab=r'$x(t)$',
               ymax1=2, ymax2=2,
               title=None ):
    '''
    外力と変位の振動波形を描画する関数
    '''
    ax1, ax2 = axs #第１軸, 第２軸
    
    ax1.plot(ts, forces,   label=inlab,  color='gray') #入力
    ax2.plot(ts, xs[:, 0], label=outlab, color='red')  #出力

    ax1.set_ylim(-ymax1, ymax1) #縦軸の範囲
    ax2.set_ylim(-ymax2, ymax2)

    ax1.set_ylabel('Input Amplitude',  fontsize=15, color='gray')
    ax2.set_ylabel('Output Amplitude', fontsize=15, color='red')

    ax1.set_xlabel('Time', fontsize=16)

    ax1.legend(fontsize=14, loc='upper left') #凡例の表示
    ax2.legend(fontsize=14, loc='upper right') #凡例の表示

    ax1.grid()   #グリッド線の表示
    
    if title is not None:
        ax1.set_title(title, fontsize=14)
        
        
def output_waves(ts, forces, xs, 
                 inlab=r'$f(t)$', outlab=r'$x(t)$', 
                 ymax1=2, ymax2=2):
    '''
    振動波形の描画を出力する関数
    '''
    fig, ax = plt.subplots(1,1, figsize=(7,3))
    axs = [ax, ax.twinx()]  #第１軸, 第２軸
    
    draw_waves(axs, ts, forces, xs, inlab, outlab, ymax1, ymax2)
    fig.tight_layout()

### 計算結果

In [ ]:
org_param = [
    1,    # m: 質量 
    0.5,  # c: 減衰係数
    2,    # k: ばね定数
    1,    # P: 外力振幅
    3,    # Om: 外力角振動数
]

X0 = [
    0,  #初期変位 
    2,  #初速度
]

ts = np.linspace(0,50,500) #時間軸を表す数列（0〜50秒を500等分）
forces, xs = Solve(funcs=org_funcs, param=org_param, x0=X0, time=ts)
output_waves(ts, forces, xs)

## 演習 5.4 ( 周波数応答の目測 )

In [ ]:
def animate_sweep_om(funcs, param, x0, time, oms):
    '''
    入力周波数による振動波形をアニメーションする関数
    '''
    fig, ax = plt.subplots(1,1, figsize=(7,3))
    axs = [ax, ax.twinx()]  #第１軸, 第２軸

    plot1 = plot2 = None #描画を収める変数（最初は空）
    def each_frame(iom):
        '''
        パラパラマンガの1コマを描く関数
        '''
        #iom番目のomegaにおける数値解
        om = oms[iom]
        forces, xs = Solve(funcs, param, x0, time, om)
        
        #現在のコマを描画
        for ax in axs:
            ax.cla() # 描画をクリア
        title = r'$\Omega$ = %.2f'%(om)
        draw_waves(axs, time, forces, xs, title=title)
        fig.tight_layout()
        
    anim = FuncAnimation(
        fig, each_frame, 
        interval=200, frames=len(oms)
    )

    rc('animation', html='jshtml')
    return anim

In [ ]:
# 多少時間がかかります．

ts = np.linspace(0, 50, 200) #時間軸を表す数列（0〜100秒を200等分）
oms = np.linspace(0.3, 3, 100)
animate_sweep_om(org_funcs, org_param, X0, ts, oms)

- 出力 $x(t)$ の振幅は，入力角振動数 $\Omega$ とともに，いったん増加してから，減少する．
- 出力の振動数は，振動が落ち着いた以降は，入力と同じになる．
- 入力と出力の波形の，位置関係（山と谷）に着目すると，
 - $\Omega$ が小さいところでは，入力と出力の山（谷）は，だいたい同じ位置．（同相という）
 - $\Omega$ を増加させると，出力の位置が，だんだん遅れてくる．
 - $\Omega$ が大きいところでは，入力の山が，出力の谷になる．（逆相という）

## 演習 5.5 ( 振幅比の数値計算 )

In [ ]:
def measure_gain_plot(funcs, param, x0, min_om, max_om, n_om, dTn=30, Tn=50):
    '''
    ゲイン線図をシミュレーション波形から測定する関数
    '''
    # 初期設定
    x0copy = x0.copy()  #初期値のコピー（元を書き換えない）
    oms    = np.linspace(min_om, max_om, n_om) #外力振動数の軸
    tn     = dTn*Tn     #時間軸の総ステップ数 = (1周期当り)*(計算する周期の個数)
    P      = param[-2]  #入力振幅＝パラメータの後ろから2番め
    
    # 測定結果を収める配列
    Rs  = np.zeros(n_om)  #変位の定常応答振幅
 
    # 入力波形の末尾4周期
    tail_len = 4*dTn           # 末尾の長さ
    tail_idx = tn - tail_len # 末尾が始まるインデックス

    for i, om in enumerate(oms):

        # 時間軸の長さを，入力の山の数で決める
        T = 2*np.pi/om                  #入力の周期
        time = np.linspace(0, T*Tn, tn) #Tn周期分の時間軸
        
        # 応答を求める
        forces, xs = Solve(funcs, param, x0copy, time, om)

        # 応答の末尾部分（≒定常応答）を抽出
        xs_     = xs[tail_idx:,:]
        
        # P-Pを測定する
        xPP  = max(xs_[:,0]) - min(xs_[:,0]) #後半のP-P（変位）

        # 振幅比をストア
        Rs[i]  = (xPP/2)/P    #(正弦波の振幅=(P-P)/2)/入力振幅

        x0copy = xs[-1,:]     #現在の終端値を，次の初期値にする
        
    return (oms, Rs)

### 計算結果

In [ ]:
oms, Rs = measure_gain_plot(org_funcs, org_param, X0, 0.1, 3, 200)

plt.plot(oms, Rs)
plt.xlabel(r'$\Omega$', fontsize=16)
plt.ylabel(r'$\bar R$', fontsize=16)
plt.grid()

## 演習 5.6 ( 相互相関関数の動作確認 )

In [ ]:
def correlate_test(xs, fs):
    '''
    相互相関関数の動作を確認する関数
    '''
    fig, axs = plt.subplots(3,1, figsize=(6,5))    
    
    K  = len(xs)
    L  = len(fs)
    
    ax = axs[0]
    ax.plot(xs, '-', label='Target')
    ax.plot(fs, '--', label='Template')
    ax.set_ylabel('value')
    ax.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
    
    # Numpy組み込みの相互相関関数
    # 　'fill'モードで（テキストの）定義通りの動作になる
    corr = np.correlate(xs, fs, 'fill')

    # Numpy配列の仕様上，corr[負]の格納はできないので，
    # corr[0] には s=0 の値ではなく，sの下限における値が格納されている．
    # そこで，定義通り，-(L-1)から始まるs軸を自作する．
    ss = np.arange(len(corr)) - (L-1)
    
    # 相互相関関数のプロット（s軸）
    ax = axs[1]
    ax.plot(ss, corr,  '-r')
    ax.set_ylabel('Cross-correlation')
    ax.set_xlabel('$s$')

    # 相互相関関数のプロット（配列の添字軸）
    ax = axs[2]
    ax.plot(corr,  '-r')
    ax.set_ylabel('Cross-correlation')
    ax.set_xlabel('Index of Numpy array')

    for ax in axs:
        ax.grid()
    
    fig.tight_layout()
    
    return fig

# テスト用の数列
xs = [0, 0, -2, 0, 1, -1, 1] #マッチング対象
fs = [0, -1, 0]              #テンプレート

fig = correlate_test(xs, fs)
fig.savefig("corr_test.pdf") #テキスト用

- 相互相関関数は $s=1$ で最大になっています．
- テンプレート（点線）を，$s=1$ だけシフト（右へ）すると，確かに対象と重なります．
- 2番めによいのが $s=4$ で，確かに波形が少し似ています．

In [ ]:
# 対象とテンプレートの立場を逆転

fig = correlate_test(fs, xs)

- 今度は，相互相関関数が $s=-1$ で最大になっています．
- テンプレート（点線）を，その分シフト（左へ）すると，確かに対象と重なります．

## 演習 5.7 ( 位相差の数値計算 )

### 準備（位相差を推定するユーザ関数）

- 相互相関関数を使った方法が，ネットでちらほらヒットするので，ちとやってみる．

In [ ]:
def estimate_delay(ts, target, template):
    '''
    2つの波形の違いが最小（＝相互相関関数が最大）
    となる遅れ時間を求める関数
    '''
    # 相互相関関数の計算
    # Numpy仕様: len(corrs) = len(target) + len(template) - 1
    corrs_full = np.correlate(target, template, 'full')
    
    # template由来の境界効果部分（先頭）を除く
    corrs = corrs_full[len(template)-1:]

    # 相互相関関数が最大値となるインデックス
    i_shift = corrs.argmax()

    if i_shift >= 0:
        t_delay = -ts[i_shift] #遅れが負（位相差に合わせる）
    else:
        t_delay = ts[-i_shift]
        
    return t_delay

def test_estimate_delay():
    '''
    テスト用の関数
    '''
    # サンプル波形
    sample_phi = -np.pi/2

    # 対象のテスト信号
    targ_ts  = np.linspace(0, 8*np.pi, 801)
    target   = np.sin(targ_ts + sample_phi)

    # テンプレート信号
    temp_ts  = np.linspace(0, 8*np.pi, 801)
    template = np.sin(temp_ts)

    plt.figure()
    plt.plot(temp_ts, template, '.', label='template')
    plt.plot(targ_ts, target,   '-', label='target')
    plt.legend()
    
    delay_ts = targ_ts
    t_delay = estimate_delay(delay_ts, target, template)
    
    print('t_delay = ',t_delay)

    plt.figure()
    plt.plot(temp_ts-t_delay, template, '.', label='template')
    plt.plot(targ_ts,         target,   '-', label='target')
    plt.legend()
    
test_estimate_delay()

### 本題（位相線図の数値計算）

In [ ]:
def measure_phase_plot(funcs, param, x0, min_om, max_om, n_om, dTn=30, Tn=50):
    '''
    位相線図をシミュレーション波形から測定する関数
    '''
    # 初期設定
    x0copy  = x0.copy() #初期値のコピー（元を書き換えない）
    oms     = np.linspace(min_om, max_om, n_om) #外力振動数の軸
    tn      = dTn*Tn  #時間軸の総ステップ数 = (1周期当り)*(計算する周期の個数)

    # 入力波形の末尾4周期
    tail_len = 4*dTn          # 末尾の長さ
    tail_idx = tn - tail_len  # 末尾が始まるインデックス

    # 位相差の測定結果を収める配列
    phi_x  = np.zeros(n_om) #変位
        
    for i, om in enumerate(oms):
        
        # 時間軸の長さを，入力の山の数で決める
        T = 2*np.pi/om                  #入力の周期
        time = np.linspace(0, T*Tn, tn) #Tn周期分の時間軸
        
        # 応答を求める
        forces, xs = Solve(funcs, param, x0copy, time, om)

        # 応答の末尾部分（≒定常応答）を抽出
        forces_ = forces[tail_idx:]  #入力
        x_      = xs[tail_idx:,0]    #出力（変位）
        
        # 変位の位相差を推定する
        # forces_を基準波形に，x_がどれだけ遅れるか
        t_delay = estimate_delay(time, x_, forces_)
        phi      = om * t_delay #外力の位相＝om*時間
        phi_x[i] = phi
        
        x0copy = xs[-1,:] #現在の終端値を，次の初期値にする

    def reshape(phis):
        '''
        位相差を 0 〜 -pi の連続関数に整形する関数
        '''
        #0 〜 -pi の範囲に折りたたむ
        phis = np.fmod(phis, np.pi)
    
        #段差を無くす
        unit = np.pi
        prev_phi = phis[0]
        for i, phi in enumerate(phis):
            err = prev_phi-phi
            err_n = np.floor(err/unit)
            if err_n > 0:
                phis[i] = phi - err_n * unit
                
            prev_phi = phis[i]
        
        return phis

    phi_x  = reshape(phi_x)
        
    return (oms, phi_x)

### 位相線図の計算結果

In [ ]:
oms, phi_x = measure_phase_plot(
    org_funcs, org_param, X0, 0.1, 3, n_om=200, dTn=200)

# n_om, dTn を増やすと，グラフは滑らかになりますが，時間が掛かります

In [ ]:
plt.plot(oms, phi_x)
plt.xlabel(r'$\Omega$', fontsize=16)
plt.ylabel(r'$\phi$', fontsize=16)
plt.yticks(
    np.linspace(0,-np.pi,5),
    [r'$0$',r'$-\pi/4$',r'$-\pi/2$',r'$-3\pi/4$',r'$-\pi$']
)
plt.xlim([0,3])
plt.grid()

- 入力振動数 $\Omega$ が低いと，位相差 $\phi$ は 0 に近くなります．
 - 出力波形の山と谷が，入力波形のそれと同期する．（同相）
- 入力振動数 $\Omega$ が高いと，位相差 $\phi$ は $-\pi$ に近くなります．
 - 出力波形の山と谷が，入力波形と互い違いになる．（逆相）

## 演習 5.8 ( ガタのある調和励振系の周波数応答 )

### ガタのある調和励振系の運動方程式

$m\ddot x + c\dot x + k\,F_{\mathrm{gata}}(x,\eta) =P\sin(\Omega t),\quad$ 
$F_{\mathrm{gata}}(x,\eta):=\begin{cases}
 x       & (x < 0)
 \\
 0       & (0 \leq x< \eta)
 \\
 x - \eta & (\eta \leq x)
 \\
\end{cases}$

を1階化すると，

$\begin{cases}
 \dot x_1 = x_2
 \\
 \dot x_2 =  \{-k\,x_1 - c\,x_2 - k\,F_{\mathrm{gata}}(x,\eta) + P\sin(\Omega t)\}/m 
\end{cases}$

In [ ]:
def Fgata(x, eta):
    '''
    ガタのあるバネを表す関数
    '''
    if x < 0:
        F = x
    elif x < eta:  #プログラムは上から実行されるので，0 <= は省略可
        F = 0
    else:            #ここの不等式も省略可
        F = x - eta

    return F

#お試しプロット
xs = np.linspace(-1,2,100)
Fgatas = np.array([
  Fgata(x, eta=1) for x in xs  
])

plt.figure(figsize=(3,2))
plt.plot(xs, Fgatas)
plt.xlabel(r'$x$')
plt.ylabel(r'$F_{\mathrm{gata}}(x,\eta)$')
plt.grid()
#plt.savefig('Fgata.pdf', bbox_inches='tight') #テキスト用

In [ ]:
def gata_funcs():
    '''
    ガタのある調和励振系の外力と運動方程式
    '''

    def force(t, param):
        '''
        外力 f(t)
        '''
        m, c, k, eta, P, Om = param
        ft = P*sin(Om*t)
        
        return ft

    def eom(x, t, param):
        '''
        運動方程式
        '''
        m, c, k, eta, P, Om = param

        return np.array([
            x[1],
            ( -k*x[0] -c*x[1] - k*Fgata(x[0], eta) + force(t,param) )/m
        ])

    return (force, eom)

### ガタがある場合の振幅比

In [ ]:
X0 = [
    0,  #初期変位 
    2,  #初速度
]

gata_param = [
    1,    # m:   質量 
    0.02, # c:   減衰係数
    1,    # k:   ばね定数
    0.4,  # eta: ガタの大きさ
    0.1,  # P:   外力振幅
    3,    # Om:  外力角振動数
]

#以下，少々時間がかかります．

#上昇掃引
oms_u, Rs_u = measure_gain_plot(
    gata_funcs, gata_param, X0, 
    min_om=1, max_om=1.6, n_om=200, Tn=150
)
#下降掃引
oms_d, Rs_d = measure_gain_plot(
    gata_funcs, gata_param, X0, 
    min_om=1.6, max_om=1, n_om=200, Tn=150
)

In [ ]:
plt.plot(oms_u, Rs_u, label='sweep up')
plt.plot(oms_d, Rs_d, label='sweep down')
plt.xlabel(r'$\Omega$', fontsize=16)
plt.ylabel(r'$\bar R$', fontsize=16)
plt.legend()
plt.grid()

### ガタの影響

#### 跳躍・履歴現象

- $\Omega$ を増加させる掃引では(up)，出力振幅は大きく成長してから，$\Omega_u\approx 1.38$ 付近で，下向きにジャンプ（急現象）する．
- $\Omega$ を減少させる掃引では(down)，小振幅のまま $\Omega_u$ を通過し，$\Omega_d\approx 1.28$ 付近で，上向きにジャンプ（急増加）する．
- このような，パラメータの変化で出力値がジャンプする現象を，**跳躍現象** という．
- 跳躍する場所が，掃引方向で異なることを，**履歴現象** という．

#### 出力振幅の初期値依存性

- $\Omega=1.3$〜$1.4$の付近では，掃引方向によって，共振曲線の高さが異なる．
- すなわち，各$\Omega$について，2種類の出力振幅が存在する．
- どちらの振幅が出現するかは，初期値に依存する．

In [ ]:
ts = np.linspace(0,300,1000) #時間軸を表す数列
gata_param[-1] = 1.38 #共振曲線が二重になるOmega

# ある初期値
X0a = [0, 2]
forces, xs = Solve(gata_funcs, gata_param, X0a, ts)
output_waves(ts, forces, xs, ymax1=0.12, ymax2=5)

In [ ]:
# 別の初期値
X0b = [0, 0]
forces, xs = Solve(gata_funcs, gata_param, X0b, ts)
output_waves(ts, forces, xs, ymax1=0.12, ymax2=5)

### ガタがある場合の位相差

- 正弦波を入力していますが，非線形系なので，出力波形は正弦波ではなくなります．
- 入力と出力の波形が異なるので，以下は，あくまで近似的な，数値計算の結果となります．
- ちなみに，数学的に理論計算する方法もありますが，やはり近似解しか出せません．

In [ ]:
# 少々時間がかかります

#上昇掃引
oms_u, phis_u = measure_phase_plot(
    gata_funcs, gata_param, X0, 1, 1.6, n_om=200, Tn=150, dTn=200)
#下降掃引
oms_d, phis_d = measure_phase_plot(
    gata_funcs, gata_param, X0, 1.6, 1, n_om=200, Tn=150, dTn=200)

In [ ]:
plt.plot(oms_u, phis_u, label='sweep up')
plt.plot(oms_d, phis_d, label='sweep down')
plt.xlabel(r'$\Omega$', fontsize=16)
plt.ylabel(r'$\phi$', fontsize=16)
plt.yticks(
    np.linspace(0,-np.pi,5),
    [r'$0$',r'$-\pi/4$',r'$-\pi/2$',r'$-3\pi/4$',r'$-\pi$']
)
plt.legend()
plt.grid()

位相差も，共振曲線と同じ場所でジャンプしています．